In [45]:
import pickle
import pandas as pd


qp = "queries/"
embeddings_path = "../app/embeddings"
models_path = "../app/models"

with open(f"{embeddings_path}/embeddings_corpus_movie", "rb") as f:
    embeddings = pickle.load(f)

with open(f"{models_path}/sentence_similarity_model", "rb") as f:
    model_1 = pickle.load(f)

In [46]:
def read(typ):
    return pd.read_csv(f"{qp}{typ}.tsv", sep="\t")

In [47]:
from sklearn.metrics.pairwise import cosine_distances,euclidean_distances
def give_suggestion(user_input:str, model, k:int=5):
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = model.encode(user_input)
    stock_similarity = []
    stock_suggest_k = []
    # on calcule les distances cosinus
    for embed,title,syn  in embeddings:
        result = (cosine_distances([embed],[embed_user]),title)
        #result = (euclidean_distances([embed],[embed_user]),title)
        # elles sont stockés dans la liste sous forme de tuple contenant la distance cosinus et le titre
        stock_similarity.append(result)
    # on trie (ordre croissant) pour avoir le synopsis le plus proche de l'input de l'usr
    ma_liste_triee = sorted(stock_similarity, key=lambda x: x[0])[:10]
    # afficher les premiers résultats etc - on peut changer cette partie en fonction de ce qu'on veut afficher
    #(idem pour le return)
    #for i, (x, y) in enumerate(ma_liste_triee[:5]):
        #print(f"Suggestion {i+1}: {y} - distance {x}")
        #stock_suggest_k.append(y)
    return ma_liste_triee #ou stock_suggest_k ou stock_similarity[:k]

In [58]:
desc = read("desc")

syns = read("syns")

query = read("query")

dfs = {'descriptions':desc,'synopsis':syns,'queries':query}

In [61]:
def eval(typ,df):
    score = 0
    for index, row in df.iterrows():
        title = row['title']
        syn = row['synopsis']
        suggestions = [res[1] for res in give_suggestion(syn, model_1)]
        with open(f"results/{typ}/{title}.txt", "w") as f:
            for res in suggestions:
                print(res, file=f)
        if title in suggestions:
            score+=1
    score = score/len(df)

    return score


In [62]:
for k,df in dfs.items():
    print(eval(k,df))

0.2777777777777778
0.5263157894736842
0.47058823529411764
